In [8]:
%run ~/glre/relmapping/annot_ce/notebooks/annot__init__.ipynb

3.5.1 |Continuum Analytics, Inc.| (default, Jun 15 2016, 15:32:45) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
os.getcwd(): /mnt/b2/home1/ahringer/fnc21/relmapping


In [9]:
def mp(fp): return os.path.join('annot_ce', 'metrics_exon', fp)

fp_atac = 'annot_ce/accessible_sites_ce.tsv'
df_atac = pd.read_csv(fp_atac, sep='\t')
l_atac_peak_pos = df_atac[['start', 'end']].mean(axis=1).map(int)
df_atac.head()

,chrom,concave_start,concave_end,mode,wt_ya_rep1_score,wt_ya_rep2_score,wt_ya_prp1_score,wt_ya_prp2_score,glp1_ya_rep1_score,glp1_ya_rep2_score,glp1_ya_prp1_score,glp1_ya_prp2_score,min_rank,start,end,wt_ya_globalIDR,glp1_ya_globalIDR,max_globalIDR
0,I,1853,2073,1954,70.953622,91.569783,82.288248,85.444344,24.319110,26.719965,25.715143,25.791774,15732.0,1879,2030,4.537940,2.024222,4.537940
1,I,3777,4041,3937,4872.445780,6185.208044,5691.562684,5697.772603,1789.116031,3081.296592,2528.127365,2594.018281,17.0,3862,4013,5.000000,5.000000,5.000000
2,I,11158,11499,11270,227.133825,232.716522,232.729857,228.530643,401.094166,368.999985,375.656917,388.186372,6221.0,11195,11346,5.000000,5.000000,5.000000
3,I,14902,15147,15075,51.754613,74.749065,71.679453,60.633056,37.247275,39.822124,38.999408,38.571533,18453.0,15000,15151,3.541128,3.092428,3.541128
4,I,15352,15544,15449,387.337816,330.166113,358.911265,344.146515,325.352257,391.857245,339.118038,391.052672,3624.0,15374,15525,5.000000,5.000000,5.000000


In [10]:
# df_exon1 / df_exon2: first/other exons, coordinates fixed to represent 5' ends
#NAMES_EXON = ('chrom', 'start', 'end', 'transcript_id', 'score', 'strand', 'gene_id', 'gene_biotype', 'display_id')
l_adj_ = [
    'chrom', 'start', 'end', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'Name', 
    'exon_start', 'exon_end', 'utr_start', 'utr_end', 'aoe_end', 'aoe_start', 'gene_biotype', 
    'exon_id', 'gene_id', 'init_cutoff_pos', #'locus_id', 
]

df_pass1_exon1_fwd = read_gffbed('annot_ce/canonical_geneset/elegans.transcripts.annot_pass1_exon1_fwd_utradj.bed')[l_adj_]
df_pass1_exon1_rev = read_gffbed('annot_ce/canonical_geneset/elegans.transcripts.annot_pass1_exon1_rev_utradj.bed')[l_adj_]
df_pass1_exon2_fwd = read_gffbed('annot_ce/canonical_geneset/elegans.transcripts.annot_pass1_exon2_fwd.bed')
df_pass1_exon2_rev = read_gffbed('annot_ce/canonical_geneset/elegans.transcripts.annot_pass1_exon2_rev.bed')
df_pass2_exon1_fwd = read_gffbed('annot_ce/canonical_geneset/elegans.transcripts.annot_pass2_exon1_fwd.bed')
df_pass2_exon1_rev = read_gffbed('annot_ce/canonical_geneset/elegans.transcripts.annot_pass2_exon1_rev.bed')


df_pass1_exon1_fwd["utr_end"] = df_pass1_exon1_fwd["utr_end"].astype(str).astype(float)
df_pass1_exon1_fwd["start"] = df_pass1_exon1_fwd["start"].astype(str).astype(float)
df_pass1_exon1_fwd["end"] = df_pass1_exon1_fwd["end"].astype(str).astype(float)

df_pass1_exon1_rev["utr_start"] = df_pass1_exon1_rev["utr_start"].astype(str).astype(float)
df_pass1_exon1_rev["start"] = df_pass1_exon1_rev["start"].astype(str).astype(float)
df_pass1_exon1_rev["end"] = df_pass1_exon1_rev["end"].astype(str).astype(float)

df_pass1_exon1_fwd.loc[(df_pass1_exon1_fwd['utr_end'] > df_pass1_exon1_fwd['start'] + 251) & (df_pass1_exon1_fwd['utr_end'] - df_pass1_exon1_fwd['start'] < 1000), 'end'] = df_pass1_exon1_fwd['utr_end']
df_pass1_exon1_fwd.loc[(df_pass1_exon1_fwd['utr_end'] < df_pass1_exon1_fwd['start'] + 251) | (df_pass1_exon1_fwd['utr_end'] - df_pass1_exon1_fwd['start'] > 1000), 'end'] = df_pass1_exon1_fwd['start'] + 251

df_pass1_exon1_rev.loc[(df_pass1_exon1_rev['utr_start'] > df_pass1_exon1_rev['end'] - 251) & (df_pass1_exon1_rev['utr_start'] - df_pass1_exon1_rev['end'] < 1000), 'start'] = df_pass1_exon1_rev['utr_start']
df_pass1_exon1_rev.loc[(df_pass1_exon1_rev['utr_start'] < df_pass1_exon1_rev['end'] - 251) | (df_pass1_exon1_rev['utr_start'] - df_pass1_exon1_rev['end'] > 1000), 'start'] = df_pass1_exon1_rev['end'] - 251


# Set 1st exon matching area to TSS + 250bp
#df_pass1_exon1_fwd['end'] = df_pass1_exon1_fwd['start'] + 1 + 250
#df_pass1_exon1_rev['start'] = df_pass1_exon1_rev['end'] - 1 - 250

print(len(df_pass1_exon1_fwd), len(df_pass1_exon1_rev),
      len(df_pass1_exon2_fwd), len(df_pass1_exon2_rev),
      len(df_pass2_exon1_fwd), len(df_pass2_exon1_rev),
)

23075 21855 140124 131302 808 757


In [11]:
df_pass1_exon1_fwd['start'] = df_pass1_exon1_fwd['start'].astype(np.int64)
df_pass1_exon1_fwd['end'] = df_pass1_exon1_fwd['end'].astype(np.int64)
#df_pass1_exon1_fwd['utr_end'] = df_pass1_exon1_fwd['utr_end'].astype(np.int64)

df_pass1_exon1_rev['start'] = df_pass1_exon1_rev['start'].astype(np.int64)
df_pass1_exon1_rev['end'] = df_pass1_exon1_rev['end'].astype(np.int64)
#df_pass1_exon1_rev['utr_start'] = df_pass1_exon1_rev['utr_start'].astype(np.int64)

In [12]:
# Match hypersensitive sites to closest first/other exon
def df_closest(df_a, df_b, b_prefix, **kwargs):
    df_a_sort = df_a
    df_b_sort = df_b.sort_values(list(df_b.columns[:3]))
    fn_ = BedTool.from_dataframe(df_a).closest(BedTool.from_dataframe(df_b_sort).fn, **kwargs).fn
    names_ = list(itertools.chain(df_a.columns.values,
        ['%s_%s' % (b_prefix, col) for col in df_b.columns.values],
        ['%s_distance' % (b_prefix)]
    ))
    df_ = pd.read_csv(fn_, names=names_, sep='\t')
    return df_[names_[len(df_a.columns):]]

# Closest exon1 -- protein_coding & pseudogene
df_ = pd.concat([df_atac['chrom'], l_atac_peak_pos, l_atac_peak_pos + 1], axis=1).copy()
df_pass1_exon1_fwd_out_ = df_closest(df_, df_pass1_exon1_fwd, 'pass1_exon1', D='ref', t='first', iu=True)
df_pass1_exon1_rev_out_ = df_closest(df_, df_pass1_exon1_rev, 'pass1_exon1', D='ref', t='last', id=True)

# Closest exon2 can be both upstream or downstream
df_pass1_exon2_fwd_out_ = df_closest(df_, df_pass1_exon2_fwd, 'pass1_exon2', D='ref', t='first')
df_pass1_exon2_rev_out_ = df_closest(df_, df_pass1_exon2_rev, 'pass1_exon2', D='ref', t='last')

# Closest exon2 can be both upstream or downstream (downstream only)
df_pass1_exon2_downstream_fwd_out_ = df_closest(df_, df_pass1_exon2_fwd, 'pass1_exon2_downstream', D='ref', t='first', iu=True)
df_pass1_exon2_downstream_rev_out_ = df_closest(df_, df_pass1_exon2_rev, 'pass1_exon2_downstream', D='ref', t='last', id=True)

# Closes pass2/exon1 -- tRNA, snoRNA, rRNA, miRNA, etc
df_pass2_exon1_fwd_out_ = df_closest(df_, df_pass2_exon1_fwd, 'pass2_exon1', D='ref', t='first', iu=True)
df_pass2_exon1_rev_out_ = df_closest(df_, df_pass2_exon1_rev, 'pass2_exon1', D='ref', t='last', id=True)

df_exon_fwd = pd.concat([df_pass1_exon1_fwd_out_, df_pass1_exon2_fwd_out_, df_pass1_exon2_downstream_fwd_out_, df_pass2_exon1_fwd_out_], axis=1)
df_exon_rev = pd.concat([df_pass1_exon1_rev_out_, df_pass1_exon2_rev_out_, df_pass1_exon2_downstream_rev_out_, df_pass2_exon1_rev_out_], axis=1)

df_exon_fwd.to_csv(mp('closest_exon_fwd.tsv'), header=True, index=False, sep='\t')
df_exon_rev.to_csv(mp('closest_exon_rev.tsv'), header=True, index=False, sep='\t')

#l_ = ['pass1_exon1_gene_id', 'pass1_exon1_locus_id', 'pass1_exon1_distance']
l_ = ['pass1_exon1_gene_id', 'pass1_exon1_distance']
write_gffbed(mp('closest_pass1_exon1_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass1_exon1_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass1_exon1_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass1_exon1_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)

#l_ = ['pass1_exon2_gene_id', 'pass1_exon2_locus_id', 'pass1_exon2_distance']
l_ = ['pass1_exon2_gene_id', 'pass1_exon2_distance']
write_gffbed(mp('closest_pass1_exon2_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass1_exon2_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass1_exon2_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass1_exon2_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)

#l_ = ['pass1_exon2_downstream_gene_id', 'pass1_exon2_downstream_locus_id', 'pass1_exon2_downstream_distance']
l_ = ['pass1_exon2_downstream_gene_id', 'pass1_exon2_downstream_distance']
write_gffbed(mp('closest_pass1_exon2_downstream_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass1_exon2_downstream_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass1_exon2_downstream_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass1_exon2_downstream_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)

#l_ = ['pass2_exon1_gene_id', 'pass2_exon1_locus_id', 'pass2_exon1_distance']
l_ = ['pass2_exon1_gene_id', 'pass2_exon1_distance']
write_gffbed(mp('closest_pass2_exon1_fwd.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_fwd['pass2_exon1_Name'],
    strand = '+',
    attr = df_exon_fwd[l_], 
)
write_gffbed(mp('closest_pass2_exon1_rev.bed'),
    chrom = df_atac['chrom'],
    start = df_atac['start'],
    end = df_atac['end'],
    name = df_exon_rev['pass2_exon1_Name'],
    strand = '-',
    attr = df_exon_rev[l_], 
)